# Expected Completion

In [32]:
import pandas as pd
import numpy as np

In [33]:
plays = pd.read_csv("data/pass_receiver.csv")
pff = pd.read_csv("data/pffScoutingData.csv")


read in weekly tracking files and concatenate into one tracking file, then delete weeklies

In [34]:
week1 = pd.read_csv("data/week1.csv")
week2 = pd.read_csv("data/week2.csv")
week3 = pd.read_csv("data/week3.csv")
week4 = pd.read_csv("data/week4.csv")
week5 = pd.read_csv("data/week5.csv")
week6 = pd.read_csv("data/week6.csv")
week7 = pd.read_csv("data/week7.csv")
week8 = pd.read_csv("data/week8.csv")
tracking = pd.concat([week1,week2,week3,week4,week5,week6,week7,week8])


In [35]:
#save memory
del week1,week2,week3,week4,week5,week6,week7,week8

In [36]:
tracking.columns

Index(['gameId', 'playId', 'nflId', 'frameId', 'time', 'jerseyNumber', 'team',
       'playDirection', 'x', 'y', 's', 'a', 'dis', 'o', 'dir', 'event'],
      dtype='object')

add unique identifier of each play and frame (some playIds repeat over games)

In [37]:
tracking["comb_id"] = tracking["gameId"].astype("str")+" "+tracking["playId"].astype("str")
pff["comb_id"] = pff["gameId"].astype("str")+" "+pff["playId"].astype("str")
plays["comb_id"] = plays["gameId"].astype("str")+" "+plays["playId"].astype("str")
tracking["comb_and_frame"] = tracking["gameId"].astype("str")+" "+tracking["playId"].astype("str") + " "+tracking["frameId"].astype("str")

isolate frames at time of pass

In [38]:
#isolate just forward passes and leave out a certain frame (duplicate forward pass for a play)
just_forward_passes = tracking[(tracking["event"]=="pass_forward") & (tracking["comb_and_frame"]!="2021091203 3740 32")& (tracking["team"]!="football")]
#bring in receiver info from plays
tracking_plays =just_forward_passes.merge(plays[["comb_id","possessionTeam","is_pass_failed","PassReceiver_nflId"]], how="left",on="comb_id" )
#and qb info from pff
tracking_merged_all = tracking_plays.merge(pff[["comb_id","nflId","pff_role"]], how = "left",on=["comb_id","nflId"])

In [39]:
len(just_forward_passes)

166034

In [40]:
len(tracking_plays)

166276

isolate tracking to just QB, OL, receiver and defense

In [41]:
tracking_players_filtered = tracking_merged_all[(tracking_merged_all["nflId"]==tracking_merged_all["PassReceiver_nflId"])|(tracking_merged_all["pff_role"]=="Pass")|(tracking_merged_all["pff_role"]=="Pass Block")|(tracking_merged_all["team"]!=tracking_merged_all["possessionTeam"])]